In [1]:
import requests
import io
from PIL import Image
import base64
from dash import Dash, html, dcc, Output, Input

# Initialize Dash app
app = Dash(__name__)
server = app.server  # For deployment

# NASA GIBS WMS endpoint
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi"

# Bounding box for your Brazil region (twice as wide)
BBOX = "-31,-85,15,-32"

# Target layers
LAYERS = [
    "MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual",
    "MODIS_Aqua_L4_LAI_8Day",
    "MODIS_Aqua_L3_NDVI_16Day",
    "MODIS_Aqua_L4_Net_Photosynthesis_8Day",
    "MODIS_Aqua_L4_FPAR_8Day"
]

# Readable layer labels for display
LAYER_LABELS = {
    "MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual": "Land Cover (Annual)",
    "MODIS_Aqua_L4_LAI_8Day": "Leaf Area Index (8-day)",
    "MODIS_Aqua_L3_NDVI_16Day": "NDVI (16-day)",
    "MODIS_Aqua_L4_Net_Photosynthesis_8Day": "Net Photosynthesis (8-day)",
    "MODIS_Aqua_L4_FPAR_8Day": "Fraction of Photosynthetically Active Radiation (8-day)"
}

def get_image_base64(layer, date_str):
    """Fetch WMS image from NASA GIBS and encode to base64."""
    params = {
        "service": "WMS",
        "version": "1.3.0",
        "request": "GetMap",
        "layers": layer,
        "style": "default",
        "format": "image/png",
        "transparent": "false",
        "crs": "EPSG:4326",
        "time": date_str,
        "bbox": BBOX,
        "width": "694",
        "height": "495",
    }

    r = requests.get(URL, params=params)
    r.raise_for_status()
    img = Image.open(io.BytesIO(r.content))
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    encoded = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/png;base64,{encoded}"

# Years of available MODIS/Aqua/Combined data
YEARS = list(range(2005, 2025))

# App Layout
app.layout = html.Div([
    html.H3("🌎 NASA MODIS Land & Vegetation Data Viewer", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Year:"),
        dcc.Slider(
            min=min(YEARS),
            max=max(YEARS),
            step=1,
            value=min(YEARS),
            marks={y: str(y) for y in YEARS[::2]},  # label every 2 years
            id="year-slider"
        )
    ], style={"margin": "25px"}),

    html.Div([
        html.Label("Select Data Layer:"),
        dcc.Slider(
            min=0,
            max=len(LAYERS) - 1,
            step=1,
            value=0,
            marks={i: LAYER_LABELS[LAYERS[i]].split()[0] for i in range(len(LAYERS))},
            id="layer-slider"
        )
    ], style={"margin": "25px"}),

    html.Div(id="status-label", style={"margin": "15px", "fontWeight": "bold"}),

    html.Img(
        id="landcover-image",
        style={"width": "700px", "border": "1px solid #ccc", "display": "block", "margin": "auto"}
    )
])

# Callback for dynamic updates
@app.callback(
    [Output("landcover-image", "src"),
     Output("status-label", "children")],
    [Input("year-slider", "value"),
     Input("layer-slider", "value")]
)
def update_image(year, layer_index):
    layer = LAYERS[layer_index]
    date_str = f"{year}-01-01"
    try:
        print(f"Fetching {layer} for {date_str}")
        img_src = get_image_base64(layer, date_str)
        label = f"🛰 {LAYER_LABELS[layer]} — Year: {year}"
        return img_src, label
    except Exception as e:
        return None, f"⚠️ Error fetching data for {LAYER_LABELS[layer]} ({year}): {e}"

if __name__ == "__main__":
    app.run(debug=True)


Fetching MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual for 2005-01-01
Fetching MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual for 2017-01-01
Fetching MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual for 2022-01-01
Fetching MODIS_Aqua_L4_Net_Photosynthesis_8Day for 2022-01-01
Fetching MODIS_Aqua_L4_Net_Photosynthesis_8Day for 2005-01-01
Fetching MODIS_Aqua_L4_Net_Photosynthesis_8Day for 2024-01-01
Fetching MODIS_Aqua_L4_LAI_8Day for 2024-01-01
Fetching MODIS_Aqua_L4_LAI_8Day for 2007-01-01
Fetching MODIS_Aqua_L4_LAI_8Day for 2006-01-01
Fetching MODIS_Aqua_L4_LAI_8Day for 2005-01-01
Fetching MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual for 2005-01-01
